In [22]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementClickInterceptedException, JavascriptException, NoSuchElementException
import time
from datetime import date, datetime, timedelta 
import numpy as np
import pandas as pd
import os
import re
from functions import allMondays, intHolidayClosures, create_trips, country_holidays

In [23]:
#constants
URL = 'https://camper.fti.de/'
DELAY = 3
PAGE_LOAD_ID = "SearchMask"
COOKIE_ID = '//*[@id="uc-center-container"]/div[2]/div/div[1]/div/button'
COOKIE_JS_PATH = 'document.querySelector("#usercentrics-root").shadowRoot.querySelector("#uc-center-container > div.sc-gWXbKe.fSIThp > div > div.sc-cxpSdN.ihpUDX > div > button")'
NOTIFICATION_JS_PATH = 'document.querySelector("#pid-14572 > div.cleverpush-confirm.cleverpush-confirm-modal.cleverpush-confirm-windows.cleverpush-confirm-chrome.cleverpush-confirm-chrome.cleverpush-confirm-1 > div.cleverpush-confirm-wrap > div.cleverpush-clearfix.cleverpush-clearfix-bottom > div.cleverpush-confirm-buttons > button.cleverpush-confirm-btn.cleverpush-confirm-btn-deny")'
POPUP_JS_PATH = 'document.querySelector("#trboModule_12920_202268_88_container > div.trbo-closeButton.trbo-theme-0.trboType-icon > i")'
CITY = 'Los Angeles'
CITY_JS_PATH = 'document.querySelector("#SearchMask > div.flexContainer.location > div:nth-child(1) > div:nth-child(1) > div > div > input")'
CAL_ELEMS_XPATH = "//span[contains(@class, 'cell day')]"
PU_CALENDAR_JS_PATH = 'document.querySelector("#SearchMask > div:nth-child(2) > div:nth-child(1) > div > div > div:nth-child(1) > input")'
DO_CALENDAR_JS_PATH = "document.querySelector('#SearchMask > div:nth-child(2) > div:nth-child(2) > div > div > div:nth-child(1) > input')"
DO_MONTH_JS_PATH = "document.querySelector('#SearchMask > div:nth-child(2) > div:nth-child(2) > div > div > div:nth-child(2) > header > span.day__month_btn.up')"
PU_MONTH_TEXT_CLASS_NAME = "vdp-datepicker__calendar"
PU_MONTH_NEXT_BTN_CLASS_NAME = "span[class = 'next']"
DO_MONTH_TEXT_XPATH = "//*[@id='SearchMask']/div[2]/div[2]/div/div/div[2]/header/span[3]"
SUBMIT_BUTTON_ID = "SubmitButton"
LOAD_MORE_RESULTS = "document.querySelector('#ResultList > button')"
# PICKUP_LOC_ID = 'pickUpLocation'
# PICKUP_LOC_XPATH = "//select[@id = 'pickUpLocation]"

In [25]:
def get_proxies():

    options = webdriver.ChromeOptions()
    #maximize argument
    options.add_argument("start-maximized")

    #removes "chrome is being controlled by automated software"
    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)

    s=Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=s, chrome_options= options)
    driver.get("https://sslproxies.org/")

    #scroll to the table
    driver.execute_script("return arguments[0].scrollIntoView(true);", WebDriverWait(driver, 20).until(EC.visibility_of_element_located((By.XPATH, "//table[@class='table table-striped table-bordered']//th[contains(., 'IP Address')]"))))

    ips = [my_elem.get_attribute("innerHTML") for my_elem in WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@class='table table-striped table-bordered']//tbody//tr/td[position() = 1]")))]
    ports = [my_elem.get_attribute("innerHTML") for my_elem in WebDriverWait(driver, 5).until(EC.visibility_of_all_elements_located((By.XPATH, "//table[@class='table table-striped table-bordered']//tbody//tr/td[position() = 2]")))]

    proxies = []
    for i in range(0, len(ips)):
        proxies.append(ips[i]+':'+ports[i])
    #print(proxies)
    
    driver.quit()

    return proxies

proxies = get_proxies()


proxies

C:\Users\mriedeman\AppData\Local\Temp\ipykernel_10356\3865060908.py:12: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(service=s, chrome_options= options)


['8.213.128.6:8080',
 '200.105.215.18:33630',
 '157.230.241.133:33273',
 '185.125.125.157:80',
 '68.183.242.248:3128',
 '176.58.112.123:1080',
 '124.106.226.240:3128',
 '178.79.138.253:8080',
 '20.54.56.26:8080',
 '172.105.107.25:999',
 '177.12.238.1:3128',
 '128.199.234.76:8080',
 '170.187.141.197:8080',
 '45.169.162.1:3128',
 '35.187.4.12:8888',
 '177.12.238.100:3128',
 '43.158.209.151:8888',
 '173.255.209.155:1080',
 '14.139.109.149:3128',
 '201.91.82.155:3128',
 '35.233.30.201:3128',
 '185.157.1.249:3128',
 '45.160.92.9:8080',
 '103.159.168.80:3128',
 '51.159.162.151:80',
 '157.100.12.138:999',
 '103.152.232.122:3125',
 '157.100.53.102:999',
 '103.247.121.114:8080',
 '96.95.164.41:3128',
 '103.111.214.106:3129',
 '159.138.255.141:8080',
 '45.79.111.38:9994',
 '49.0.2.242:8090',
 '47.253.214.60:57114',
 '41.65.252.103:1981',
 '51.159.120.129:8001',
 '149.129.213.200:8080',
 '157.230.34.152:41885',
 '5.189.184.6:80',
 '192.53.163.144:3128',
 '139.162.182.54:49165',
 '85.159.214.61:10

In [24]:


options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)


prefs = {"profile.default_content_setting_values.notifications" : 2}
options.add_experimental_option("prefs",prefs)

s=Service(ChromeDriverManager().install())
browser = webdriver.Chrome(service=s, options = options)

browser.get(URL)

try:
    myElem = WebDriverWait(browser, DELAY).until(EC.presence_of_element_located((By.ID, PAGE_LOAD_ID)))
    print ("Page is ready!")
except TimeoutException:
    print ("Couldn't find element")

[WDM] - Downloading: 100%|██████████| 6.21M/6.21M [00:01<00:00, 5.33MB/s]


Page is ready!


In [ ]:
browser.switch_to.alert.text
browser.switch_to.alert.accept()
browser.switch_to.alert.dismiss()

In [3]:
def create_browser(URL, ELEMENT_ID):

    """Creates the browser and loads the page given url and webelement"""

    options = webdriver.ChromeOptions()

    
    #options.add_argument(f'--proxy-server={proxies[-1]}')
    s=Service(ChromeDriverManager().install())
    browser = webdriver.Chrome(service=s)

    browser.get(URL)

    try:
        myElem = WebDriverWait(browser, DELAY).until(EC.presence_of_element_located((By.ID, ELEMENT_ID)))
        print ("Page is ready!")
    except TimeoutException:
        print ("Couldn't find element")

    return browser


browser = create_browser(URL, PAGE_LOAD_ID)

[WDM] - Downloading: 100%|██████████| 6.21M/6.21M [00:01<00:00, 5.38MB/s]


Page is ready!


In [25]:
#accept cookies

#not sure if this will work
#WebDriverWait(browser, 5, poll_frequency=5).until(EC.alert_is_present(), 'Timed out waiting for Accept Cookies alert to appear')



browser.execute_script(f"return {COOKIE_JS_PATH}").click()

In [26]:
#deny news

#not sure if this will work
#WebDriverWait(browser, 15, poll_frequency=5).until(EC.alert_is_present(), 'Timed out waiting for News alert to appear')
browser.execute_script(f"return {NOTIFICATION_JS_PATH}").click()

In [33]:
#close popup
#WebDriverWait(browser, 15, poll_frequency=5).until(EC.alert_is_present(), 'Timed out waiting for News alert to appear')
browser.execute_script(f"return {POPUP_JS_PATH}").click()

In [6]:
#use if running code below more than once

browser.execute_script(f"return {CITY_JS_PATH}").click()
time.sleep(1)
browser.execute_script(f"return {CITY_JS_PATH}").clear()

In [6]:
browser.execute_script(f"return {CITY_JS_PATH}").send_keys(f'{CITY}')
time.sleep(1)
browser.execute_script(f"return {CITY_JS_PATH}").send_keys(Keys.RETURN)

In [7]:
#simulate a single trip
german_months = {
    1 : 'Jan',
    2 : 'Feb',
    3 : 'Mär',
    4 : 'Apr',
    5 : 'Mai',
    6 : 'Jun',
    7 : 'Jul',
    8 : 'Aug',
    9 : 'Sep',
    10: 'Okt',
    11: 'Nov',
    12: 'Dez'

}

start_date = date(2022, 11, 14)
end_date = start_date + timedelta(17)
trip = {}
trip['start_date'] = start_date
trip['start_month'] = german_months[start_date.month]
trip['start_day'] = start_date.day
trip['end_date'] = end_date
trip['end_day'] = end_date.day
trip['end_month'] = german_months[end_date.month]
trip['end_day']
trip['city'] = CITY

In [8]:
#open the pickup month calendar
browser.execute_script(f"return {PU_CALENDAR_JS_PATH}").click()

In [10]:
#select the pickup month
cal_month = browser.find_element(By.CLASS_NAME, f"{PU_MONTH_TEXT_CLASS_NAME}").text[2:5]

while cal_month != trip['start_month']:
    
    browser.find_element(By.CSS_SELECTOR, f"{PU_MONTH_NEXT_BTN_CLASS_NAME }").click()
    time.sleep(1)
    cal_month = browser.find_element(By.CLASS_NAME, f"{PU_MONTH_TEXT_CLASS_NAME}").text[2:5]
    print(cal_month)
    

In [11]:
#click on the pickup day
cal_elems = browser.find_elements(By.XPATH, f"{CAL_ELEMS_XPATH}")
cal_elems_dict = {}
for ele in cal_elems:

    if ele.text.isdigit():

        cal_elems_dict[ele.text] = ele
        
cal_elems_dict[str(trip['start_day'])].click()

In [12]:
#open the drop off calendar

browser.execute_script(f"return {DO_CALENDAR_JS_PATH}").click()

In [13]:
#select the drop off month
cal_month = browser.execute_script(f"return {DO_MONTH_JS_PATH}").text[:3]
while cal_month != trip['end_month']:
    browser.find_element(By.XPATH, f"{DO_MONTH_TEXT_XPATH}").click()
    #updates cal_month variable to whatever date you are currently on
    time.sleep(int(np.random.rand(1)*3))
    cal_month = browser.execute_script(f"return {DO_MONTH_JS_PATH}").text[:3]
    print(cal_month)

Dez


In [45]:
#select drop off day
cal_elems = browser.find_elements(By.XPATH, f"{CAL_ELEMS_XPATH}")
cal_elems_dict = {}
for ele in cal_elems:

    if ele.text.isdigit():

        cal_elems_dict[ele.text] = ele
        
cal_elems_dict[str(trip['end_day'])].click()

In [14]:
#click search
browser.find_element(By.ID, f"{SUBMIT_BUTTON_ID }").click()

0


In [ ]:
# #explicity wait for load
# for j in range(1)   
#     i = 0 
#     LOADING_ICON = "MainLoader"
#     while i < 5:
#         try:
#             myElem = WebDriverWait(browser, DELAY).until_not(EC.presence_of_element_located((By.ID, LOADING_ICON)))
#             print ("Page is ready!")
#         except TimeoutException:
#             print ("still Loading..")
#             i+=1
#             print("Trying again...")

#             if i == 3:
#                 browser.refresh()
#                 print("Refresshing page to try again. Connection May be slow")
#                 time.sleep(3)
        
#     if i == 5:
#         print("Could not Load page after several attempts. Refreshing the browser and beginning with the next trip.")
#         browser.get(URL)
#         continue

    

In [40]:
#ADD LOGIC TO WAIT FOR PAGE TO LOAD AND GO BACK IF THERE ARE NO RESULTS

In [17]:
#load all the results
browser.execute_script(f"return {LOAD_MORE_RESULTS}").click()

In [18]:
LISTINGS_CLASS_NAME = "resultCard"
listings = browser.find_elements(By.CLASS_NAME, f"{LISTINGS_CLASS_NAME}")
listings

[<selenium.webdriver.remote.webelement.WebElement (session="eefcb1c29749a83cafc67206c8a9dcd6", element="17c0bfaf-03a6-4648-8b87-da2f789c695a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eefcb1c29749a83cafc67206c8a9dcd6", element="67c4b55b-9c45-4825-8f87-8f368cd7b772")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eefcb1c29749a83cafc67206c8a9dcd6", element="b74ec294-76da-4aff-8f44-74d1bfb2c929")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eefcb1c29749a83cafc67206c8a9dcd6", element="2c50bde4-8830-4de6-9b64-a18ee8433cf6")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eefcb1c29749a83cafc67206c8a9dcd6", element="27d72101-b993-4cf3-b19a-aef121c54c1a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eefcb1c29749a83cafc67206c8a9dcd6", element="4bfed701-d148-4c4d-a206-f644e5ef0f51")>,
 <selenium.webdriver.remote.webelement.WebElement (session="eefcb1c29749a83cafc67206c8a9dcd6", element="fc6f0ca4-a863-4ba8-b9d8-c6

In [51]:
#listings[0].find_element(By.CLASS_NAME, "provider-data").text.split("\n")[0]

'Station Wagon'

In [57]:
#listings[0].find_element(By.CLASS_NAME, "camper-details").text

'auf Anfrage\nUnbegrenzte Freimeilen\nFahrzeugversicherung mit Selbstbeteiligung USD 3.000\nZusatzversicherung zur Rückerstattung der Selbstbeteiligung\nEUR 10 Mio. Zusatz-Haftpflichtversicherung\nZusatzfahrer\nGesamtpreis\n1171,00 EUR\n68,88 EUR pro Tag'

In [65]:
#money = listings[0].find_element(By.CLASS_NAME, "camper-details").find_element(By.CSS_SELECTOR, 'div.priceSummary.full > dl > dd').text.split('\n')

In [69]:
# import re

# '.'.join(re.findall(r'\d+', money[0]))

'1171.00'

In [19]:
trip_data = []
for card in listings:

    vehicle_class = card.find_element(By.CLASS_NAME, "provider-data").text.split("\n")[0]
    company_name = card.find_element(By.CLASS_NAME, "providerLogo").get_attribute("alt")
    price_string = card.find_element(By.CLASS_NAME, "camper-details").find_element(By.CSS_SELECTOR, 'div.priceSummary.full > dl > dd').text.split('\n')
    total_price = '.'.join(re.findall(r'\d+', price_string[0]))
    daily_price = '.'.join(re.findall(r'\d+', price_string[1]))


    trip_data.append({
    'city': trip['city'],
    'start_date':trip['start_date'],
    'end_date':trip['end_date'],
    'total_price':total_price,
    'daily_price': daily_price,
    'vehicle_class':vehicle_class,
    'company_name':company_name
    })

pd.DataFrame(trip_data)

,city,start_date,end_date,total_price,daily_price,vehicle_class,company_name
0,Los Angeles,2022-11-14,2022-12-01,816.00,74.18,Station Wagon,Travellers Autobarn
1,Los Angeles,2022-11-14,2022-12-01,1078.00,98.00,Motorhome C25,Road Bear
2,Los Angeles,2022-11-14,2022-12-01,1092.00,99.27,Motorhome M28,Mighty
3,Los Angeles,2022-11-14,2022-12-01,1111.00,101.00,Campervan B21,Road Bear
4,Los Angeles,2022-11-14,2022-12-01,1161.00,105.55,Hitop Campervan,Travellers Autobarn
5,Los Angeles,2022-11-14,2022-12-01,1161.00,105.55,Kuga Campervan,Travellers Autobarn
6,Los Angeles,2022-11-14,2022-12-01,1180.00,107.27,C Medium 24-26,Mighty
7,Los Angeles,2022-11-14,2022-12-01,1221.00,111.00,Motorhome C28,Road Bear
8,Los Angeles,2022-11-14,2022-12-01,1255.00,114.09,Motorhome Standard C25,Cruise
9,Los Angeles,2022-11-14,2022-12-01,1255.00,114.09,Motorhome Large C30,Cruise


In [20]:
trip_data_df = pd.DataFrame(trip_data)

In [81]:
URL

'https://camper.fti.de/'

In [21]:
try: 
    os.makedirs(f"../Output Data/FTI Camper/{trip['city']}")
except FileExistsError:
    pass

trip_data_df.to_csv(f"../Output Data/FTI Camper/{trip['city']}/{trip['city']} {date.today()}.csv", index=False)